In [97]:
import pandas as pd
#import matplotlib.pyplot as plt
#import matplotlib.dates as mdates
import numpy as np
from datetime import datetime
from datetime import date

### Declare User Variables

In [98]:
margin_min = 1.15 #play with these
margin_max = 1.85 #play with these

pd.options.display.max_columns = 50

### Setup ExcelWriter

In [99]:
writer = pd.ExcelWriter('outputs/BadDearItemsProject.xlsx')

### Import CSV "Products without descriptions"

In [100]:
df = pd.read_csv('inputs/InventoryList_2022-11-08.csv',header=0,dtype={'ProductCode': str,'Barcode':str})
df.head(2)

,ProductCode,Name,Category,Brand,Type,FixedAssetType,CostingMethod,Length,Width,Height,Weight,CartonLength,CartonWidth,CartonHeight,CartonInnerQuantity,CartonQuantity,CartonVolume,WeightUnits,DimensionUnits,Barcode,MinimumBeforeReorder,ReorderQuantity,DefaultLocation,LastSuppliedBy,SupplierProductCode,...,StockLocator,PurchaseTaxRule,SaleTaxRule,Status,Description,ShortDescription,Sellable,PickZones,AlwaysShowQuantity,WarrantySetupName,InternalNote,ProductionBOM,MakeToOrderBom,QuantityToProduce,IsAccountingDimensionEnabled,DimensionAttribute1,DimensionAttribute2,DimensionAttribute3,DimensionAttribute4,DimensionAttribute5,DimensionAttribute6,DimensionAttribute7,DimensionAttribute8,DimensionAttribute9,DimensionAttribute10
0,NaN,NaN,Service,NaN,Service,NaN,FIFO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,Airport,NaN,NaN,...,NaN,NaN,NaN,ACTIVE,NaN,NaN,Yes,NaN,NaN,NaN,NaN,No,No,1.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,777823111113,1 Water 355ml,Non Alcoholic Beverage,NaN,Stock,NaN,FIFO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,oz,in,777823111113,0.0,0.0,Pleasant Valley,Restaurant Depot,NaN,...,NaN,No Tax,No Tax,ACTIVE,NaN,NaN,Yes,NaN,0.0,NaN,NaN,No,No,1.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Get rid of stupid columns

In [101]:
#df.columns
len(df.columns)

94

In [102]:
df = df[['ProductCode', 'Name', 'Category', 'Type', 'Barcode',
       'MinimumBeforeReorder', 'ReorderQuantity', 'DefaultLocation',
       'LastSuppliedBy', 'SupplierProductCode', 'SupplierFixedPrice', 'PriceTier1', 'PriceTier2', 'PriceTier3',
       'PriceTier4', 'PriceTier5', 'PriceTier6', 'PriceTier7', 'PriceTier8',
       'PriceTier9', 'PriceTier10', 'AverageCost','DefaultUnitOfMeasure', 'InventoryAccount', 'RevenueAccount',
       'ExpenseAccount', 'COGSAccount', 'ProductAttributeSet',
       'AdditionalAttribute1', 'AdditionalAttribute2', 'AdditionalAttribute3',
       'AdditionalAttribute4','PurchaseTaxRule', 'SaleTaxRule', 'Status', 'Description',
       'ShortDescription', 'Sellable', 'AlwaysShowQuantity', 'InternalNote']].copy()
len(df.columns)

40

In [103]:
df[df.Category == 'Service'].to_excel(writer, sheet_name='Merges',index=False)

### Bad Categories / Category Setup

In [104]:
all_cats = df.Category.unique()

In [105]:
alcoholic_cats = ['Craft Beer',
       'Tequila', 'Bubbles', 'Whiskey Bourbon/American', 'RTD', 'Vodka',
       'Mixer', 'Whiskey Irish', 'Orange Wine', 'Red New World',
       'Whiskey Rye', 'Red Zinfandel', 'Mezcal', 'Gin', 'Liqueur',
       'Whiskey Mexican', 'Imp Beer', 'Whiskey Scotch',
       'Absinthe', 'Fortified', 'Red', 'White', 'Rum',
       'Whisky English', 'Red Old World', 'Whiskey Japanese',
       'Whiskey Canadian', 'Red Merlot', 'Rose', 'Grappa',
       'Amaro', 'Whiskey Indian',
       'Whiskey American Single Malt', 'White Sauvignon Blanc', 'Cider',
       'Brandy', 'Cachaca', 'Whiskey South African', 'Aquavit',
       'Pisco', 'Shochu', 'White Pinot Grigio', 'Dom Beer',
       'Whiskey French', 'Red Cabernet Sauvignon', 'Red Tempranillo',
       'Cognac', 'Calvados', 'Red Pinot Noir', 'Red Chianti',
       'Red Bordeaux', 'White Picpoul', 'Sotol', 'Bacanora', 'Raicilla',
       'Red Beaujolais', 'Soju', 'Mead', 'Sake',
       'Whisky German', 'Red Gamay', 'White Chenin', 'White Chardonnay',
       'White Muscadet', 'Red Blend', 'Whiskey Chech',
       'Red Montepulciano', 'Whiskey Taiwaneese', 'White Reisling',
       'White wine', 'Red Mourvedre', 'Red Malbec', 'Baijiu', 'Singani',
       'Pox', 'Charanda', 'Whiskey Australian', 'White Gewurztraminer',
       'Red Primitivo']

In [106]:
bad_categories = ['Red Zinfandel','Red','Red Merlot','White Sauvignon Blanc','White Pinot Grigio','Red Cabernet Sauvignon', 'Red Tempranillo','Red Pinot Noir', 'Red Chianti',
       'Red Bordeaux', 'White Picpoul','Red Beaujolais','Red Gamay', 'White Chenin', 'White Chardonnay','White Muscadet', 'Red Blend','Red Montepulciano','White Reisling','White wine', 'Red Mourvedre', 'Red Malbec','White Gewurztraminer','Red Primitivo']
good_cats = [x for x in all_cats if x not in bad_categories]
#good_cats

In [107]:
df[(df.Category.isin(bad_categories)) & (df.Status == "ACTIVE")].to_excel(writer, sheet_name='Bad Categories',index=False)

# Items with Bad Pricing

### Retail that NEEDS to change

In [108]:
df[((df.AverageCost > df.PriceTier1) | (df.PriceTier1 == 0.0)) & (df.Status == "ACTIVE")].to_excel(writer, sheet_name='Change Retail Prices',index=False)

### Wholesale that NEEDS to change

In [109]:
df[(df.AverageCost > df.PriceTier2) & (df.PriceTier2 != 0) & (df.Status == "ACTIVE")].to_excel(writer, sheet_name='Change Wholesale Prices',index=False)

### Craft a Margin Column

In [110]:
df['ProfitRetail'] = df.PriceTier1 - df.AverageCost
df['MarginRetail'] = df.ProfitRetail / df.PriceTier1
#df[(((df.AverageCost * margin_max) < df.PriceTier1) | ((df.AverageCost * margin_min) > df.PriceTier1)) & (df.Status == "ACTIVE")]
df[['ProductCode','Name','MarginRetail','Category','SupplierFixedPrice','PriceTier1','PriceTier2','AverageCost','Status','ProfitRetail']].to_excel(writer, sheet_name='Margin Column',index=False)

### Bad Accounting Categories

In [111]:
df[(df.InventoryAccount != '_28_: Inventory') | (df.RevenueAccount != '_1_: Sales')  |  (df.ExpenseAccount.notnull()) | (df.COGSAccount != '_26_: Cost of Goods Sold')].to_excel(writer, sheet_name='Suspect Accounting',index=False)

### Missing Additional Attributes

In [112]:
df[((df.AdditionalAttribute1.isnull()) | (df.AdditionalAttribute2.isnull())) & df.Category.isin(alcoholic_cats)].to_excel(writer, sheet_name='Additional Attributes',index=False)

### Playing Around

In [113]:
all_cats = df.Category.unique()

In [114]:
alcoholic_cats = ['Craft Beer',
       'Tequila', 'Bubbles', 'Whiskey Bourbon/American', 'RTD', 'Vodka',
       'Mixer', 'Whiskey Irish', 'Orange Wine', 'Red New World',
       'Whiskey Rye', 'Red Zinfandel', 'Mezcal', 'Gin', 'Liqueur',
       'Whiskey Mexican', 'Imp Beer', 'Whiskey Scotch',
       'Absinthe', 'Fortified', 'Red', 'White', 'Rum',
       'Whisky English', 'Red Old World', 'Whiskey Japanese',
       'Whiskey Canadian', 'Red Merlot', 'Rose', 'Grappa',
       'Amaro', 'Whiskey Indian',
       'Whiskey American Single Malt', 'White Sauvignon Blanc', 'Cider',
       'Brandy', 'Cachaca', 'Whiskey South African', 'Aquavit',
       'Pisco', 'Shochu', 'White Pinot Grigio', 'Dom Beer',
       'Whiskey French', 'Red Cabernet Sauvignon', 'Red Tempranillo',
       'Cognac', 'Calvados', 'Red Pinot Noir', 'Red Chianti',
       'Red Bordeaux', 'White Picpoul', 'Sotol', 'Bacanora', 'Raicilla',
       'Red Beaujolais', 'Soju', 'Mead', 'Sake',
       'Whisky German', 'Red Gamay', 'White Chenin', 'White Chardonnay',
       'White Muscadet', 'Red Blend', 'Whiskey Chech',
       'Red Montepulciano', 'Whiskey Taiwaneese', 'White Reisling',
       'White wine', 'Red Mourvedre', 'Red Malbec', 'Baijiu', 'Singani',
       'Pox', 'Charanda', 'Whiskey Australian', 'White Gewurztraminer',
       'Red Primitivo']

In [115]:
bad_categories = ['Red Zinfandel','Red','Red Merlot','White Sauvignon Blanc','White Pinot Grigio','Red Cabernet Sauvignon', 'Red Tempranillo','Red Pinot Noir', 'Red Chianti',
       'Red Bordeaux', 'White Picpoul','Red Beaujolais','Red Gamay', 'White Chenin', 'White Chardonnay','White Muscadet', 'Red Blend','Red Montepulciano','White Reisling','White wine', 'Red Mourvedre', 'Red Malbec','White Gewurztraminer','Red Primitivo']
good_cats = [x for x in all_cats if x not in bad_categories]
#good_cats

In [116]:
df[(df.Category.isin(bad_categories)) & (df.Status == "ACTIVE")].to_excel(writer, sheet_name='Bad Categories',index=False)

In [117]:
writer.save()

In [118]:
#Good Categories Static(?)

#['Service',
# 'Non Alcoholic Beverage',
# 'Craft Beer',
# 'Bar Tools',
# 'Tequila',
# 'Bubbles',
# 'Whiskey Bourbon/American',
# 'RTD',
# 'Vodka',
# 'Mixer',
# 'Whiskey Irish',
# 'Orange Wine',
# 'Red New World',
# 'Whiskey Rye',
# 'Mezcal',
# 'Gin',
# 'Bag',
# 'Liqueur',
# 'Whiskey Mexican',
# 'Imp Beer',
# 'Whiskey Scotch',
# 'Glassware',
# 'Absinthe',
# 'Garnish',
# 'Books',
# 'Fortified',
# 'White',
# 'Rum',
# 'Whisky English',
# 'Retail',
# 'Red Old World',
# 'Whiskey Japanese',
# 'Whiskey Canadian',
# 'Other',
# 'Rose',
# 'Grappa',
# 'Bitters',
# 'Amaro',
# 'Tobacco',
# 'Whiskey Indian',
# 'Whiskey American Single Malt',
# 'Cider',
# 'Brandy',
# 'Cachaca',
# 'Ice',
# 'Whiskey South African',
# 'Aquavit',
# 'Pisco',
# 'Shochu',
# 'Dom Beer',
# 'Snacks',
# 'Whiskey French',
# 'Fruit',
# 'Cognac',
# 'Calvados',
# 'Sotol',
# 'Bacanora',
# 'Raicilla',
# 'Liability',
# 'Soju',
# 'Mead',
# 'Sake',
# 'Whisky German',
# 'Whiskey Chech',
# 'Whiskey Taiwaneese',
# 'Baijiu',
# 'Singani',
# 'Pox',
# 'Charanda',
# 'Whiskey Australian']